In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

import tensorflow as tf
import keras
import keras.backend as K
from keras import Input, layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import ZeroPadding1D, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D, Dropout, LSTM, Bidirectional, concatenate
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import plot_model

print("Tensorflow version", tf.__version__)
print("Keras version", keras.__version__)

Tensorflow version 1.14.0
Keras version 2.2.4


Using TensorFlow backend.


In [2]:
#set global model parameters
n_epochs = 3
n_subepochs = 3

n_channels = 1
frequency = 100  #edf: 100, physionet: 200
classes = 5
epoch_length = 30

train_test_ratio = 0.7
fit_epochs = 3

# Model Building
## Refactor Model according to Paper
<b>Modifikations:</b><br>
ResNet:
 - one-dimensional operations instead of two-dimensional operations (convolution, max-pool, batch normalization)
 - Additional max-pool between the conv3 x layer and the conv4 x layer to reduce the length of the feature sequence by half.
 - Exclude the global average pool layer 
 - Add a dropout layer at the end of the convolutional layers in order to prevent overfitting with p = 0:5. </ul>
Recurrent Layers:
- Two layers of BiLSTM was adopted. 
- Hidden state size of BiLSTM in each direction and the number of the last convolutional filters was set as u = 128.

## Building Blocks

In [3]:
# Identity Block - IITNet
def identity_block_iitnet(input_x, kernel_size, filters, stage, block, branch):
    """
    Implementation of the identity block (or bottleneck block) as described in 
    https://arxiv.org/pdf/1512.03385.pdf and implemented in Keras. Altered as described 
    in Back (2019)
    
    # Arguments
        input_x: Input tensor 
        kernel_size: default 3, the kernel size of middle conv layer for the main path
        filters: list of integers, the number of filters in the conv layers at the main path
        stage: integer, current stage label, used for generating layer names
        branch: index of the input branch
        block: 'a','b'..., current block label, used for generating layer names
    
    # Returns
        Output tensor
    """
    
    #define name basis for the layers in the block
    conv_name_base = 'branch' + branch + '_res' + str(stage) + block + '_branch'
    bn_name_base = 'branch' + branch + '_bn' + str(stage) + block + '_branch'
    
    #get the filter numbers for the three conv layers
    filters1, filters2, filters3 = filters
    
    #store the input tensor for later
    x_shortcut = input_x
    
    #first component (main path)
    x = Conv1D(filters = filters1,
               kernel_size = 1,
               strides = 1,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2a')(input_x)
    x = BatchNormalization(name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #second component (main path)
    x = Conv1D(filters = filters2,
               kernel_size = kernel_size,
               strides = 1,
               padding = 'same',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2b')(x)
    x = BatchNormalization(name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #third component (main path)
    x = Conv1D(filters = filters3,
               kernel_size = 1,
               strides = 1,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2c')(x)
    x = BatchNormalization(name = bn_name_base + '2c')(x)
    
    #final step: addition of main path and recurrent shortcut, activation
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)
    
    return x

In [23]:
# Convolutional Block
def convolutional_block_iitnet(input_x, kernel_size, filters, stage, block, branch, strides=2):
    """
    Implementation of the convolutional (or standard block) as described in 
    https://arxiv.org/pdf/1512.03385.pdf and implemented in Keras. Altered as described 
    in Back (2019)
    
    # Arguments
        input_x: Input tensor 
        kernel_size: default 3, the kernel size of middle conv layer for the main path
        filters: list of integers, the number of filters in the conv layers at the main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        branch: index of the input branch
        stride: strides for the first convolutional layer in the block
    
    # Returns
        Output tensor
    """
    
    #define name basis for the layers in the block
    conv_name_base = 'branch' + branch + '_res' + str(stage) + block + '_branch'
    bn_name_base = 'branch' + branch + '_bn' + str(stage) + block + '_branch'
    
    #get the filter numbers for the three conv layers
    filters1, filters2, filters3 = filters
    
    #store the input tensor for later
    x_shortcut = input_x
    
    #first component (main path)
    x = Conv1D(filters = filters1,
               kernel_size = 1,
               strides = strides,
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2a')(input_x)
    x = BatchNormalization(name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #second component (main path)
    x = Conv1D(filters = filters2,
               kernel_size = kernel_size,
               strides = 1,
               padding = 'same',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2b')(x)
    x = BatchNormalization(name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #third component (main path)
    x = Conv1D(filters = filters3,
               kernel_size = 1,
               strides = 1,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2c')(x)
    x = BatchNormalization(name = bn_name_base + '2c')(x)
    
    #shortcut path
    x_shortcut = Conv1D(filters = filters3,
                        kernel_size = 1,
                        strides = strides,
                        kernel_initializer = 'he_normal',
                        name = conv_name_base + '1')(x_shortcut)
    x_shortcut = BatchNormalization(name = bn_name_base + '1')(x_shortcut)
    
    #final step: addition of main path and recurrent shortcut, activation
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)
    
    return x

In [5]:
def ResNet50(x_input, branch=1):
    """
    Build the ResNet50 architecture as described in https://arxiv.org/pdf/1512.03385.pdf 
    and implemented in Keras. Altered as described in Back (2019)
    
    # Arguments
        input_x: Input tensor 
        branch: index of the input branch
        
    # Returns
        Output tensor
    """
    
    branch_index = str(branch)
    
    #Zero Padding
    x = ZeroPadding1D(padding = 3, name = 'conv1_pad' + branch_index)(x_input)
    
    #Stage 1
    x = Conv1D(8, 2, #resnet: 64, 7, 
               strides = 2,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = 'conv1' + branch_index)(x)
    x = BatchNormalization(name = 'bn_conv1' + branch_index)(x)
    x = Activation('relu')(x)
    x = ZeroPadding1D(padding = 1, 
                      name = 'pool1_pad' + branch_index)(x)
    x = MaxPooling1D( 3, 
                     strides = 2)(x)
    
    print(x)
    
    #Stage 2
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [8, 8, 16], #resnet: [64, 64, 256]
                            stage = 2, 
                            block = 'a',
                            branch = branch_index,
                            strides = 1)
    x = identity_block_iitnet(input_x = x,
                                kernel_size = 3,
                                filters = [8, 8, 16], #resnet: [64, 64, 256]
                               stage = 2,
                               block = 'b',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [8, 8, 16], #resnet: [64, 64, 256]
                       stage = 2,
                       block = 'c',
                        branch = branch_index)
    
    print(x)
    
    #Stage 3
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [16, 16, 32], #resnet: [128, 128, 512]
                            stage = 3, 
                            block = 'a', 
                            strides = 2,
                            branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [16, 16, 32], #resnet: [128, 128, 512]
                       stage = 3,
                       block = 'b',
                        branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [16, 16, 32], #resnet: [128, 128, 512]
                       stage = 3,
                       block = 'c',
                        branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [16, 16, 32], #resnet: [128, 128, 512]
                       stage = 3,
                       block = 'd',
                        branch = branch_index)
    
    print(x)
    
    #Additional max-pool to reduce the length of the feature sequence by half
    x = MaxPooling1D()(x)
    
    print(x)
    
    #Stage 4
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [32, 32, 64], #resnet: [256, 256, 1024] 
                            stage = 4, 
                            block = 'a', 
                            strides = 2,
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'b',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'c',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'd',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'e',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'f',
                              branch = branch_index)
    
    print(x)
    
    #Stage 5
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [64, 64, 128], #resnet: [512, 512, 2048]
                            stage = 5, 
                            block = 'a', 
                            strides = 2,
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [64, 64, 128], #resnet: [512, 512, 2048]
                       stage = 5,
                       block = 'b',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [64, 64, 128], #resnet: [512, 512, 2048]
                       stage = 5,
                       block = 'c',
                              branch = branch_index)
    
    print(x)
    
    #dropout layer to prevent overfitting
    x = Dropout(0.5)(x)
    
    print(x)
    
    return x

## IITNet
For every subject that is to be fed into the model: Generate overlapping subbatches of L epochs
    
Put those subbatches into an array of subbatches (one for alle subjects)
    
Feed every subbatch into the model (in batches of e.g. 128 subbatches)

Jedes Subject hat eine andere Anzahl an Epochs


In [6]:
#Build the IITNet Architecture
def build_IITNet(n_epochs=1, n_subepochs=1, n_channels=1, frequency=100, classes=5, epoch_length=30):
    """
    Build the IITNet architecture as described in Back (2019)
    
    # Arguments
        n_epochs = L = number of time epochs in this subbatch to be used for classification
        n_subepochs = l = number of subparts every epoch is split into
        n_channels = used number of EEG/EOG/... channels
        frequency = number of measured values per second
        classes = number of classes (sleepphases) to classify epochs into
        epoch_length = default 30, length of one epoch in seconds
        
    # Returns
        classification model
    """
    
    #get the length of every subepoch (in seconds and in datapoints)
    subepoch_length_seconds = epoch_length / n_subepochs
    subepoch_length_points = subepoch_length_seconds * frequency
    
    #define the shape of all inputs (shape of the subepochs)    
    input_shape = (int(subepoch_length_points), n_channels)
    
    #prepare the list of feature sequences
    x_list = []
    
    ## For every subepoch a feature list is calculated in an own branch
    #get the total number of subepochs
    n_subepochs_total = n_epochs * n_subepochs
    
    #make a list of names for the input layers (one for each subepoch)
    input_layers = {}
    
    names_input_layers = []
    for i in range(n_subepochs_total):
        name = "x_input_" + str(i)
        names_input_layers.append(name)
    
    for i, input_layer in enumerate(names_input_layers):
        #create the first layer in the branch to represent the input data
        input_layers[input_layer] = Input(input_shape)
        
        print(input_layers[input_layer])  #TODO: Remove after Test
    
        #get the feature sequence for this subepoch
        x = ResNet50(input_layers[input_layer], branch=i)
        
        #put the outputs into a list
        x_list.append(x)
    
    #TODO: Bring the strings together = make a list of output sequences
    if n_subepochs_total > 1:
        x = concatenate(x_list)
    else:
        x = x_list[0]
    
    print(x)
    
    #recurrent layers
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(128))(x)
    
    #fully connected layer + classifier
    x = Dense(classes,
              activation = 'softmax',
              name = 'fc' + str(classes))(x)    
    
    #create model
    model = Model(list(input_layers.values()), x, name = 'IITNet')
    
    return model

In [24]:
model_iitnet = build_IITNet(n_epochs=n_epochs, 
                            n_subepochs=n_subepochs,
                            n_channels=n_channels,
                            frequency=frequency,
                            classes=classes,
                            epoch_length=epoch_length)
model_iitnet.summary()

Tensor("input_19:0", shape=(?, 1000, 1), dtype=float32)
Tensor("max_pooling1d_37/Squeeze:0", shape=(?, 252, 8), dtype=float32)
Tensor("activation_892/Relu:0", shape=(?, 252, 16), dtype=float32)
Tensor("activation_904/Relu:0", shape=(?, 126, 32), dtype=float32)
Tensor("max_pooling1d_38/Squeeze:0", shape=(?, 63, 32), dtype=float32)
Tensor("activation_922/Relu:0", shape=(?, 32, 64), dtype=float32)
Tensor("activation_931/Relu:0", shape=(?, 16, 128), dtype=float32)
Tensor("dropout_19/cond/Merge:0", shape=(?, 16, 128), dtype=float32)
Tensor("input_20:0", shape=(?, 1000, 1), dtype=float32)
Tensor("max_pooling1d_39/Squeeze:0", shape=(?, 252, 8), dtype=float32)
Tensor("activation_941/Relu:0", shape=(?, 252, 16), dtype=float32)
Tensor("activation_953/Relu:0", shape=(?, 126, 32), dtype=float32)
Tensor("max_pooling1d_40/Squeeze:0", shape=(?, 63, 32), dtype=float32)
Tensor("activation_971/Relu:0", shape=(?, 32, 64), dtype=float32)
Tensor("activation_980/Relu:0", shape=(?, 16, 128), dtype=float32)
T

__________________________________________________________________________________________________
branch2_bn2b_branch2c (BatchNor (None, 252, 16)      64          branch2_res2b_branch2c[0][0]     
__________________________________________________________________________________________________
branch3_bn2b_branch2c (BatchNor (None, 252, 16)      64          branch3_res2b_branch2c[0][0]     
__________________________________________________________________________________________________
branch4_bn2b_branch2c (BatchNor (None, 252, 16)      64          branch4_res2b_branch2c[0][0]     
__________________________________________________________________________________________________
branch5_bn2b_branch2c (BatchNor (None, 252, 16)      64          branch5_res2b_branch2c[0][0]     
__________________________________________________________________________________________________
branch6_bn2b_branch2c (BatchNor (None, 252, 16)      64          branch6_res2b_branch2c[0][0]     
__________

__________________________________________________________________________________________________
branch8_bn3a_branch2c (BatchNor (None, 126, 32)      128         branch8_res3a_branch2c[0][0]     
__________________________________________________________________________________________________
branch8_bn3a_branch1 (BatchNorm (None, 126, 32)      128         branch8_res3a_branch1[0][0]      
__________________________________________________________________________________________________
add_292 (Add)                   (None, 126, 32)      0           branch0_bn3a_branch2c[0][0]      
                                                                 branch0_bn3a_branch1[0][0]       
__________________________________________________________________________________________________
add_308 (Add)                   (None, 126, 32)      0           branch1_bn3a_branch2c[0][0]      
                                                                 branch1_bn3a_branch1[0][0]       
__________

activation_1000 (Activation)    (None, 126, 16)      0           branch2_bn3d_branch2a[0][0]      
__________________________________________________________________________________________________
activation_1049 (Activation)    (None, 126, 16)      0           branch3_bn3d_branch2a[0][0]      
__________________________________________________________________________________________________
activation_1098 (Activation)    (None, 126, 16)      0           branch4_bn3d_branch2a[0][0]      
__________________________________________________________________________________________________
activation_1147 (Activation)    (None, 126, 16)      0           branch5_bn3d_branch2a[0][0]      
__________________________________________________________________________________________________
activation_1196 (Activation)    (None, 126, 16)      0           branch6_bn3d_branch2a[0][0]      
__________________________________________________________________________________________________
activation

activation_1251 (Activation)    (None, 32, 32)       0           branch7_bn4b_branch2a[0][0]      
__________________________________________________________________________________________________
activation_1300 (Activation)    (None, 32, 32)       0           branch8_bn4b_branch2a[0][0]      
__________________________________________________________________________________________________
branch0_res4b_branch2b (Conv1D) (None, 32, 32)       3104        activation_908[0][0]             
__________________________________________________________________________________________________
branch1_res4b_branch2b (Conv1D) (None, 32, 32)       3104        activation_957[0][0]             
__________________________________________________________________________________________________
branch2_res4b_branch2b (Conv1D) (None, 32, 32)       3104        activation_1006[0][0]            
__________________________________________________________________________________________________
branch3_re

__________________________________________________________________________________________________
branch8_bn4d_branch2b (BatchNor (None, 32, 32)       128         branch8_res4d_branch2b[0][0]     
__________________________________________________________________________________________________
activation_915 (Activation)     (None, 32, 32)       0           branch0_bn4d_branch2b[0][0]      
__________________________________________________________________________________________________
activation_964 (Activation)     (None, 32, 32)       0           branch1_bn4d_branch2b[0][0]      
__________________________________________________________________________________________________
activation_1013 (Activation)    (None, 32, 32)       0           branch2_bn4d_branch2b[0][0]      
__________________________________________________________________________________________________
activation_1062 (Activation)    (None, 32, 32)       0           branch3_bn4d_branch2b[0][0]      
__________

add_397 (Add)                   (None, 32, 64)       0           branch6_bn4f_branch2c[0][0]      
                                                                 activation_1213[0][0]            
__________________________________________________________________________________________________
add_413 (Add)                   (None, 32, 64)       0           branch7_bn4f_branch2c[0][0]      
                                                                 activation_1262[0][0]            
__________________________________________________________________________________________________
add_429 (Add)                   (None, 32, 64)       0           branch8_bn4f_branch2c[0][0]      
                                                                 activation_1311[0][0]            
__________________________________________________________________________________________________
activation_922 (Activation)     (None, 32, 64)       0           add_301[0][0]                    
__________

branch3_bn5c_branch2a (BatchNor (None, 16, 64)       256         branch3_res5c_branch2a[0][0]     
__________________________________________________________________________________________________
branch4_bn5c_branch2a (BatchNor (None, 16, 64)       256         branch4_res5c_branch2a[0][0]     
__________________________________________________________________________________________________
branch5_bn5c_branch2a (BatchNor (None, 16, 64)       256         branch5_res5c_branch2a[0][0]     
__________________________________________________________________________________________________
branch6_bn5c_branch2a (BatchNor (None, 16, 64)       256         branch6_res5c_branch2a[0][0]     
__________________________________________________________________________________________________
branch7_bn5c_branch2a (BatchNor (None, 16, 64)       256         branch7_res5c_branch2a[0][0]     
__________________________________________________________________________________________________
branch8_bn

In [8]:
#! pydot has to be installed
plot_model(model_iitnet, to_file='iitnet_model_plot.png', show_shapes=True, show_layer_names=True)

## Test Model with Artificial Dataset
Data Generation, Data Preprocessing, Model Training

In [ ]:
def create_subbatches_dataset(subjects_train, n_epochs, n_subepochs, frequency, epoch_length=30):
    """
    Data Preprocessing: From the input array of subjects, create an array of subbatches, containing
    subsets of timeepochs of the length L for model training.
    One subbatch is used to feed the model one time.
    
    # Arguments
        subjects_train = array of subjects
        n_epochs = L = number of time epochs in the batch to be used for classification
        n_subepochs = l = number of subparts every epoch is to be split into
        frequency = number of measured values per second
        epoch_length = default 30, length of one epoch in seconds
        
    # Returns
        array of subbatches
    """
    
    subbatches = []
    #create subbatches for every subject
    ##NOTE: for now, a subject is just two lists of measured values and their labels
    for subject in subjects_train:
        subbatch = create_subbatches_subject(subject, n_epochs, n_subepochs, frequency, epoch_length)
        subbatches.append(subbatch)
    
    subbatches = np.asarray(subbatches)
    print(subbatches.shape)
    subbatches.shape[0]
    subbatches = subbatches.reshape((subbatches.shape[0]*subbatches.shape[1], subbatches.shape[2]))
    print(subbatches.shape)
    
    return subbatches
    

def create_subbatches_subject(subject, n_epochs, n_subepochs, frequency, epoch_length):
    """
    Data Preprocessing: From the input subject, create an array of subbatches, containing
    subsets of timeepochs of the length L for model training.
    
    # Arguments
        subject = array of measured timesequences and their labels
        n_epochs = L = number of time epochs in the batch to be used for classification
        n_subepochs = l = number of subparts every epoch is to be split into
        frequency = number of measured values per second
        epoch_length = default 30, length of one epoch in seconds
        
    # Returns
        array of subbatches
    """
    x_subject = subject[0]
    y_subject = subject[1]
    
    #number of time sequences = epochs in the subjects
    n_epochs_subject = x_subject.shape[0]
    print("n_epochs_subject: ", str(n_epochs_subject))
    
    #make subbatches of the length L
    n_subbatches = n_epochs_subject - (n_epochs - (n_epochs - 1))
    subbatches_subject = []
    print("n_subbatches: ", n_subbatches)
    
    for i in range(n_subbatches):
        print("Subbatch i: ", str(i))
        #create subbatches with the length of L (n_epochs) epochs
        one_subbatch_x = x_subject[i : n_epochs+i]
        
        #divide every epoch in the subbatch into l subepochs
        subbatch_subepochs_x = create_subepochs(one_subbatch_x, n_subepochs, frequency, epoch_length)     
        
        #label for the whole subbatch is the label of the last epoch
        one_subbatch_y = y_subject[n_epochs]
        
        one_subbatch = [subbatch_subepochs_x, one_subbatch_y]
        
        subbatches_subject.append(one_subbatch)
        
    subbatches_subject = np.asarray(subbatches_subject)
        
    return subbatches_subject
    

def create_subepochs(epochs, n_subepochs, frequency, epoch_length):
    """
    Data Preprocessing: From the input subbatch of epochs, create an array of l subepochs.
    
    # Arguments
        epochs = subbatch of epochs
        n_subepochs = l = number of subparts every epoch is to be split into
        frequency = number of measured values per second
        epoch_length = default 30, length of one epoch in seconds
        
    # Returns
        array of subepochs for the subbatch
    """
    
    subepochs_list = []
    
    #jede Epoche ist eine Liste von frequency x epoch_length Werten
    #total number of datapoints in one epoch
    epoch_length_points = epoch_length * frequency
    #length of one subepoch in datapoints
    subepoch_length_points = epoch_length_points / n_subepochs
    
    #look at one epoch
    for i, epoch in enumerate(epochs):
        print(epoch, "\n______")
        
        #divide this one epoch into l subepochs
        for j in range(n_subepochs):
            print("Subepoch ", str(j))
            
            #get the start and end point of the subepoch
            start_point = j*subepoch_length_points
            end_point = (j+1) * subepoch_length_points
            
            print(start_point, end_point)
            
            if end_point > epoch_length_points:
                break
            one_subepoch = epoch[int(start_point) : int(end_point)]
            
            subepochs_list.append(one_subepoch)
        
        #divide into l subepochs of the length subepoch_length_points
        #start_point = i*subepoch_length_points  #0, 3, 6
        #end_point = (i+1) * subepoch_length_points    #2, 5, 8
        
        #print(start_point, end_point)
        
        #if end_point > epoch_length_points: #len(epoch):
        #    break
        #one_subepoch = epoch[int(start_point) : int(end_point)]
        
        #subepochs_list.append(one_subepoch)
        
    subepochs_list = np.asarray(subepochs_list)
        
    return subepochs_list

In [ ]:
#try with an artificial dataset
## dataset enthält 2 subjects. subject enthält 4 epochen. jede Epoche hat 8 Werte. 1 channel.
## L = 2, l = 2
x_s1 = np.array([
      [[1],[2],[3],[4],[1],[2],[3],[4],[1],[2],[3],[4],[1],[2],[3],[4]],
      [[5],[6],[7],[8],[5],[6],[7],[8],[5],[6],[7],[8],[5],[6],[7],[8]],
      [[9],[10],[11],[12],[9],[10],[11],[12],[9],[10],[11],[12],[9],[10],[11],[12]],
      [[13],[14],[15],[16],[13],[14],[15],[16],[13],[14],[15],[16],[13],[14],[15],[16]]
     ])
x_s2 = np.array([
      [[17],[18],[19],[20],[17],[18],[19],[20],[17],[18],[19],[20],[17],[18],[19],[20]],
      [[21],[22],[23],[24],[21],[22],[23],[24],[21],[22],[23],[24],[21],[22],[23],[24]],
      [[25],[26],[27],[28],[25],[26],[27],[28],[25],[26],[27],[28],[25],[26],[27],[28]],
      [[29],[30],[31],[32],[29],[30],[31],[32],[29],[30],[31],[32],[29],[30],[31],[32]]
     ])
y_s1 = np.array(
     [
      [1],
      [0],
      [1],
      [0]
     ])
y_s2 = np.array([
      [1],
      [0],
      [1],
      [0]         
     ])

print(x_s1.shape)
print(x_s2.shape)
print(y_s1.shape)
print(y_s2.shape)

In [ ]:
subject_1 = [x_s1, y_s1]
subject_2 = [x_s2, y_s2]
subject = [subject_1, subject_2]

In [ ]:
subbatches_array = create_subbatches_dataset(subjects_train=subject, 
                                             n_epochs=2, n_subepochs=2, frequency=1, epoch_length=8)

In [ ]:
#subbatches

print("number subbatches: ", len(subbatches_array))
print("one subbatch:\n ", subbatches_array[0])
print("Label for this subbatch: ", subbatches_array[0][1])
print("\nSubepochs in the subbatch: ", len(subbatches_array[0][0]))
print("One Subepoch:\n ", subbatches_array[0][0][0])

#subbatches_array[0][0].reshape((8,2))

In [ ]:
#separate x and y values of the subbatches
subbatches_x = []
subbatches_y = []

for subbatch in subbatches_array:
    subbatches_x.append(subbatch[0])
    subbatches_y.append(subbatch[1])

subbatches_x = np.array(subbatches_x)
subbatches_y = np.array(subbatches_y)

print(subbatches_x.shape)
print(subbatches_x)
print(subbatches_y.shape)
print(subbatches_y)    #fertig

In [ ]:
#umformen der x-Werte
n_subepochs = 4
values_subepochs = []
    
for subepoch_nr in range(n_subepochs):
    subepoch = []
    for subbatch in subbatches_x:
        subepoch.append(subbatch[subepoch_nr])
    values_subepochs.append(subepoch)

In [ ]:
print(np.asarray(values_subepochs[1]))

### Training

In [ ]:
model_iitnet_test = build_IITNet(n_epochs=2, n_subepochs=2, n_channels=1, frequency=1, classes=1, epoch_length=8)
model_iitnet_test.summary()

In [ ]:
model_iitnet_test.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

input_x = [[subbatches_array[0][0][0]], 
           [subbatches_array[0][0][1]],
           [subbatches_array[0][0][2]], 
           [subbatches_array[0][0][3]]]
model_iitnet_test.fit(input_x, [1])

In [ ]:
model_iitnet_test.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

input_x = [[subbatches_array[0][0][0], subbatches_array[1][0][0]], 
           [subbatches_array[0][0][1], subbatches_array[1][0][1]],
           [subbatches_array[0][0][2], subbatches_array[1][0][2]], 
           [subbatches_array[0][0][3], subbatches_array[1][0][3]]]
model_iitnet_test.fit(input_x, [1, 0])

In [ ]:
model_iitnet_test.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])

model_iitnet_test.fit(values_subepochs, subbatches_y)

In [ ]:
input_x_test = [[subbatches_array[2][0][0]], 
                [subbatches_array[2][0][1]],
                [subbatches_array[2][0][2]], 
                [subbatches_array[2][0][3]]]
model_iitnet_test.evaluate(input_x_test, [0])

# Data Preparation
## EEG Data
Datensatz = Array von Subjects <br>
Subject = Array mit zwei Einträgen: Data und Labels <br>
Data: Array von Messdaten; Shape: (n_Epochen, Werte pro Epoche, Kanäle) <br>
Labels: Array von Labels; Shape: (Epochen, 5)

## Data Loading
Data Access, Selection of the EEG channel

### Data Access
Access the preprocessed, feature engineered and prepared for training data from the directory

In [9]:
#setup the file paths (sleep-edf)
base_path = "C:/Users/sa6pr7/PycharmProjects/medical-icu/src/data/deep_sleep/processed/"

files_eng_x = []
files_eng_y = []

files_eng_x.append(base_path + "577000099446A/C4001E0/0_data_x_sa6pr7_raw_1.npy")
files_eng_x.append(base_path + "577000099446A/C4002E0/0_data_x_sa6pr7_raw_1.npy")

files_eng_y.append(base_path + "577000099446A/C4001E0/0_data_y_sa6pr7_raw_1.npy")
files_eng_y.append(base_path + "577000099446A/C4002E0/0_data_y_sa6pr7_raw_1.npy")

In [ ]:
#setup the file paths (physionet)
base_path = "C:/Users/sa6pr7/PycharmProjects/medical-icu/src/data/physionet_challenge/"

files_eng_x = []
files_eng_y = []

files_eng_x.append(base_path + "577000099446A/tr03-0005/0_data_x_el9ba5_engKind_1.npy")
files_eng_x.append(base_path + "577000099446A/tr03-0029/0_data_x_el9ba5_engKind_1.npy")

files_eng_y.append(base_path + "577000099446A/tr03-0005/0_data_y_el9ba5_engKind_1.npy")
files_eng_y.append(base_path + "577000099446A/tr03-0029/0_data_y_el9ba5_engKind_1.npy")

In [10]:
data_x = []
data_y = []

for path in files_eng_x:
    data = np.load(path)
    data_x.append(np.asarray(data))
    
for path in files_eng_y:
    data = np.load(path)
    data_y.append(np.asarray(data))

data_x = np.asarray(data_x)
data_y = np.asarray(data_y)

In [11]:
print("Got", str(len(data_x)), "samples")
print("Shape of first sample:", data_x[0][:,0,:].shape)
print("Data has", len(data_x[0][0,:,0]), "channel(s)")

Got 2 samples
Shape of first sample: (2650, 3000)
Data has 1 channel(s)


### Create Samples

In [12]:
subjects = []
for index, subject_x in enumerate(data_x):
    #reformat to account for single channel
    single_channel_x = subject_x[: , 0, :]
    #create the subject
    subject = [single_channel_x, data_y[index]]
    subjects.append(subject)

### Train-Test-Split
Split the subjects into training and test dataset

In [13]:
#train-test ratio = 0.7 (see top of page)
n_subjects = len(subjects)
n_train_subjects = int(n_subjects * train_test_ratio)

subjects_train = subjects[:n_train_subjects]
subjects_test = subjects[n_train_subjects:]

print("Training samples:", n_train_subjects, "  Test samples:", (n_subjects-n_train_subjects))

Training samples: 1   Test samples: 1


## Data Preprocessing

Datenformat prepared data (code basis): (X, 2, 6000)<br>
 \= (Anzahl Epochen, Anzahl der Messpunkte pro Epoche, Kanäle)<br>

<b>Zielformat:</b><br>
(n_subepochen_pro_batch, n_alle_subbatches, n_punkte_pro_subepoche, n_kanäle)<br>
\= (9, 855, 2000, 1) <br>
<i>bei L=3 und l=3, Länge der Subepochen = 30 sec, Frequenz = 200Hz</i><br>

Subepoche 1  ---  Subbatch 1  ---  Messpunkt 1  ---  Kanal 1<br>
  \                      --- Kanal 2<br>
  \                      --- ...<br>
  \              --- Messpunkt 2 ---  Kanal 1<br>
  \                      --- Kanal 2<br>
  \                      --- ...<br>
  \              --- Messpunkt 2000 ---  ...<br>
 \           ---  Subbatch 2  ---  Messpunkt 1  ---  Kanal 1<br>
  \                      --- ...<br>
  \              --- Messpunkt 2000 ---  ...<br>
 \           ---  Subbatch ...  ---  ...

Subepoche 2  ---  Subbatch 1  ---  Messpunkt 1  ---  Kanal 1<br>
  \                      --- ...<br>
  \              --- Messpunkt ... ---  ...<br>
 \           ---  Subbatch ...  ---  ...<br>
 ...<br>
Subepoche 9  ---  Subbatch 1 ---  ...

### Methods: Subbatch Creation

In [14]:
#Ein subbatch besteht aus einem array, der die x-werte aller subepochen umfasst,
# und einem y-array, der das Label für diesen Subbatch darstellt

def subjects_to_subbatches(subjects, n_epochs, n_subepochs, frequency, epoch_length):
    #get the subbatches for every subject (every subject has epochs and labels)
    all_subjects_subbatches = []

    for subject in subjects:
        #subject = Abfolge von X Epochen + deren Labels
        #übergebe subject an Methode epochs_to_subbatches, erhalte array von subbatches
        subject_subbatches = epochs_to_subbatches(subject, n_epochs, n_subepochs, frequency, epoch_length)

        all_subjects_subbatches.append(subject_subbatches)
        
    all_subjects_subbatches = np.asarray(all_subjects_subbatches)

    #append subbatches of all subjects to each other (flatten/concatenate)
    if all_subjects_subbatches.shape[0] == 1:
        all_subjects_subbatches = np.squeeze(all_subjects_subbatches, axis=0)
    else:
        all_subjects_subbatches = np.concatenate(all_subjects_subbatches[:])

    #return an array of subbatches with their labels
    return all_subjects_subbatches


def epochs_to_subbatches(subject, n_epochs, n_subepochs, frequency, epoch_length):
#split the array of epochs into subbatches with L epochs each
    subbatches = []
    
    n_epochs_subject = len(subject[0])
    print("Subject contains", str(n_epochs_subject), "epochs")
    n_subbatches_total = n_epochs_subject - (n_epochs - 1)
    print("Splitting subject into", str(n_subbatches_total), "subbatches")
    
    for i in range(n_subbatches_total):
        subbatch_x = subject[0][i : (n_epochs+i)]
        #split every epoch of the subbatch into l subepochs
        subbatch_x_split = np.asarray(subbatch_x_to_subepochs(subbatch_x, n_epochs, n_subepochs, 
                                                              frequency, epoch_length))
        
        #label of each subbatch is label of the last epoch in the subbatch
        subbatch_y = subject[1][n_epochs + (i-1)]
        subbatch = [subbatch_x_split, subbatch_y]
        
        subbatches.append(subbatch)
        
    #return: Liste von subbatches (jedes subbatch enthält L * l subepochen + 1 Label)
    return(subbatches)

def subbatch_x_to_subepochs(subbatch_x, n_epochs, n_subepochs, frequency, epoch_length):
#split the input array of epochs into l subepochs each
    subepochs = []
    
    datapoints_per_epoch = frequency * epoch_length
    datapoints_per_subepoch = datapoints_per_epoch / n_subepochs
    
    for i, epoch in enumerate(subbatch_x):
        #split into l subepochs
        for j in range(n_subepochs):
            start_index = int(datapoints_per_subepoch * j)
            end_index = int(datapoints_per_subepoch * (j+1))
            subepoch = epoch[start_index : end_index]
            
            subepochs.append(subepoch)
    
    return subepochs

### Method: Transformation to Model Input (x)

In [15]:
def subbatches_to_modeldata_x(subbatches_x, n_epochs, n_subepochs):
    
    #part the data into L strands to feed into the model with L*l inputs
    subepoch_strands = []
    n_subepochs_subbatch = n_epochs * n_subepochs
    
    for subepoch_idx in range(n_subepochs_subbatch):
        strand = []
        
        for subbatch_idx in range(len(subbatches_x)):
            strand.append(subbatches_x[subbatch_idx][subepoch_idx])
            
        strand = np.asarray(strand)
        
        n_subbatches = strand.shape[0]
        datapoints_subepoch = strand.shape[1]
        strand = strand.reshape((n_subbatches, datapoints_subepoch, n_channels))
        
        subepoch_strands.append(strand)

    #return a list of the length L
    return subepoch_strands

## Entire method for Data Preprocessing

In [16]:
def prepare_data_for_model(subjects, n_epochs, n_subepochs, frequency, epoch_length):
    
    #transform the subjects into an array of subbatches, each of the length L epochs / L*l subepochs
    subbatches_all = subjects_to_subbatches(subjects, n_epochs, n_subepochs, frequency, epoch_length)
    print("Got total of", str(subbatches_all.shape[0]), "subbatches.")
    
    #split into data and labels arrays
    subbatches_all_x = subbatches_all[:,0]
    subbatches_all_y = subbatches_all[:,1]
    
    #part the data into L*l strands, each to be fed as input into the iitnet model
    data_x = subbatches_to_modeldata_x(subbatches_all_x, n_epochs, n_subepochs)
    
    #correct the label array
    data_y = np.asarray([x for x in subbatches_all_y])
    
    return data_x, data_y

In [17]:
train_data_x, train_data_y = prepare_data_for_model(subjects_train, n_epochs, n_subepochs, frequency, epoch_length)

Subject contains 2650 epochs
Splitting subject into 2648 subbatches
Got total of 2648 subbatches.


In [18]:
test_data_x, test_data_y = prepare_data_for_model(subjects_test, n_epochs, n_subepochs, frequency, epoch_length)

Subject contains 2830 epochs
Splitting subject into 2828 subbatches
Got total of 2828 subbatches.


# Model Training

In [19]:
model_iitnet = build_IITNet(n_epochs=n_epochs, 
                            n_subepochs=n_subepochs, 
                            n_channels=n_channels, 
                            frequency=frequency, 
                            classes=classes, 
                            epoch_length=epoch_length)
model_iitnet.summary()

Tensor("input_10:0", shape=(?, 1000, 1), dtype=float32)
Tensor("max_pooling1d_19/Squeeze:0", shape=(?, 252, 8), dtype=float32)
Tensor("activation_451/Relu:0", shape=(?, 252, 16), dtype=float32)
Tensor("activation_463/Relu:0", shape=(?, 126, 32), dtype=float32)
Tensor("max_pooling1d_20/Squeeze:0", shape=(?, 63, 32), dtype=float32)
Tensor("activation_481/Relu:0", shape=(?, 32, 64), dtype=float32)
Tensor("activation_490/Relu:0", shape=(?, 16, 128), dtype=float32)
Tensor("dropout_10/cond/Merge:0", shape=(?, 16, 128), dtype=float32)
Tensor("input_11:0", shape=(?, 1000, 1), dtype=float32)
Tensor("max_pooling1d_21/Squeeze:0", shape=(?, 252, 8), dtype=float32)
Tensor("activation_500/Relu:0", shape=(?, 252, 16), dtype=float32)
Tensor("activation_512/Relu:0", shape=(?, 126, 32), dtype=float32)
Tensor("max_pooling1d_22/Squeeze:0", shape=(?, 63, 32), dtype=float32)
Tensor("activation_530/Relu:0", shape=(?, 32, 64), dtype=float32)
Tensor("activation_539/Relu:0", shape=(?, 16, 128), dtype=float32)
T

__________________________________________________________________________________________________
branch6_bn3b_branch2b (BatchNor (None, 126, 16)      64          branch6_res3b_branch2b[0][0]     
__________________________________________________________________________________________________
branch7_bn3b_branch2b (BatchNor (None, 126, 16)      64          branch7_res3b_branch2b[0][0]     
__________________________________________________________________________________________________
branch8_bn3b_branch2b (BatchNor (None, 126, 16)      64          branch8_res3b_branch2b[0][0]     
__________________________________________________________________________________________________
activation_456 (Activation)     (None, 126, 16)      0           branch0_bn3b_branch2b[0][0]      
__________________________________________________________________________________________________
activation_505 (Activation)     (None, 126, 16)      0           branch1_bn3b_branch2b[0][0]      
__________

__________________________________________________________________________________________________
activation_467 (Activation)     (None, 32, 32)       0           branch0_bn4b_branch2a[0][0]      
__________________________________________________________________________________________________
activation_516 (Activation)     (None, 32, 32)       0           branch1_bn4b_branch2a[0][0]      
__________________________________________________________________________________________________
activation_565 (Activation)     (None, 32, 32)       0           branch2_bn4b_branch2a[0][0]      
__________________________________________________________________________________________________
activation_614 (Activation)     (None, 32, 32)       0           branch3_bn4b_branch2a[0][0]      
__________________________________________________________________________________________________
activation_663 (Activation)     (None, 32, 32)       0           branch4_bn4b_branch2a[0][0]      
__________

activation_823 (Activation)     (None, 32, 32)       0           branch7_bn4f_branch2b[0][0]      
__________________________________________________________________________________________________
activation_872 (Activation)     (None, 32, 32)       0           branch8_bn4f_branch2b[0][0]      
__________________________________________________________________________________________________
branch0_res4f_branch2c (Conv1D) (None, 32, 64)       2112        activation_480[0][0]             
__________________________________________________________________________________________________
branch1_res4f_branch2c (Conv1D) (None, 32, 64)       2112        activation_529[0][0]             
__________________________________________________________________________________________________
branch2_res4f_branch2c (Conv1D) (None, 32, 64)       2112        activation_578[0][0]             
__________________________________________________________________________________________________
branch3_re

In [20]:
model_iitnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

W0429 14:38:27.771034 11748 deprecation_wrapper.py:119] From C:\Users\sa6pr7\Anaconda3\envs\mlbase\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [21]:
model_iitnet.fit(train_data_x, train_data_y, epochs=fit_epochs, validation_split=0.2)

W0429 14:38:37.076518 11748 deprecation.py:323] From C:\Users\sa6pr7\Anaconda3\envs\mlbase\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2118 samples, validate on 530 samples
Epoch 1/3
2118/2118 [==============================] - 1055s 498ms/step - loss: 1.0909 - acc: 0.5335 - val_loss: 1.5657 - val_acc: 0.0019
Epoch 2/3
2118/2118 [==============================] - 668s 316ms/step - loss: 0.8995 - acc: 0.6133 - val_loss: 2.3290 - val_acc: 0.0000e+00
Epoch 3/3
2118/2118 [==============================] - 668s 315ms/step - loss: 0.8145 - acc: 0.6506 - val_loss: 2.1858 - val_acc: 0.0075


In [22]:
model_iitnet.evaluate(test_data_x, test_data_y)

2828/2828 [==============================] - 400s 142ms/step


[2.644832904133129, 0.4557991513437058]